# Convolutional Neural Network

### Summary

This notebook contains an CNN implementations. 
completee

###  Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear, ReLU, Sequential, Conv2d, Module, Softmax, BatchNorm2d
from torchsummary import summary

### ResBlock Class

Complete

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, num_hidden, size):
        super().__init__()
        self.conv1 = nn.Conv2d(num_hidden, num_hidden, kernel_size=size, padding='same')
        self.bn1 = nn.BatchNorm2d(num_hidden)
        self.conv2 = nn.Conv2d(num_hidden, num_hidden, kernel_size=size, padding='same')
        self.bn2 = nn.BatchNorm2d(num_hidden)
        
    def forward(self, x):
        residual = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        x = F.relu(x)
        return x
    

### Net Class

Complete

In [ ]:
class Net(Module):
    def __init__(self, size, action_size, num_resBlocks, num_hidden):
        super().__init__()

        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.startBlock = nn.Sequential(
            nn.Conv2d(3, num_hidden, kernel_size=size, padding='same'),
            nn.BatchNorm2d(num_hidden),
            nn.ReLU()
        )
        
        self.backBone = nn.ModuleList(
            [ResBlock(num_hidden, size) for i in range(num_resBlocks)]
        )
        
        self.policyHead = nn.Sequential(
            nn.Conv2d(num_hidden, 32, kernel_size=size, padding='same'),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32 * size * size, action_size),
            nn.Softmax(dim=1)
        )
        
        self.valueHead = nn.Sequential(
            nn.Conv2d(num_hidden, size, kernel_size=size, padding='same'),
            nn.BatchNorm2d(size),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(size * size * size, 1),
            nn.Tanh()
        )

        self.to(self.device)
    
    def forward(self, x):
        x = self.startBlock(x)
        for resBlock in self.backBone:
            x = resBlock(x)
        policy = self.policyHead(x)
        value = self.valueHead(x)
        return policy, value
    